In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import time

In [9]:
url = 'https://assist.org/transfer/results?year=75&institution=79&agreement=124&agreementType=from&viewAgreementsOptions=true&view=agreement&viewBy=major&viewSendingAgreements=false&viewByKey=75%2F124%2Fto%2F79%2FMajor%2F23d79a84-d16c-4b58-7dee-08dcb87d5deb'

In [10]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=True)
page = await browser.new_page()
await page.goto(url, wait_until='networkidle')
await page.wait_for_selector('.rowReceiving')

<JSHandle preview=JSHandle@node>

In [11]:
content = await page.content()
soup = BeautifulSoup(content, 'html.parser')
courses = soup.find_all('div', class_='rowReceiving')
print(f"Found {len(courses)} receiving course blocks.")

Found 11 receiving course blocks.


In [14]:
print(soup.prettify())

<!DOCTYPE html>
<html data-critters-container="" lang="en">
 <head>
  <script async="" src="https://www.google-analytics.com/analytics.js">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-BK5B9XCQZP&amp;cx=c&amp;gtm=45He55u1v9126837703za200&amp;tag_exp=101509157~103116026~103200004~103233427~103351866~103351868" type="text/javascript">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-WZQHKT5">
  </script>
  <script>
   //setup GA dataLayer object
  window.dataLayer = window.dataLayer || [];
  </script>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
  new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
  j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
  'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
  })(window,document,'script','dataLayer','GTM-WZQHKT5');
  </script>
  <!-- End Google 

In [ ]:
async def scrape():
    url = 'https://assist.org/transfer/results?year=75&institution=79&agreement=124&agreementType=from&viewAgreementsOptions=true&view=agreement&viewBy=major&viewSendingAgreements=false&viewByKey=75%2F124%2Fto%2F79%2FMajor%2F23d79a84-d16c-4b58-7dee-08dcb87d5deb'

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until='networkidle')
        await page.wait_for_selector('.rowReceiving')

        content = await page.content()
        soup = BeautifulSoup(content, 'html.parser')

        courses = soup.find_all('div', class_='rowReceiving')
        for receiving_course in courses:
            receiving_course_num = receiving_course.find_next(class_='prefixCourseNumber')
            receiving_course_title = receiving_course.find_next(class_='courseTitle')
            sending = receiving_course.find_next_sibling(class_='rowSending')
            try:
                articulation = sending.find('awc-articulation-sending').find('div', class_='view_sending__content')
            except:
                return None
            else:
                conjunction = articulation.find('awc-view-conjunction')
                first_sending_num = sending.find_next(class_='prefixCourseNumber')
                first_sending_title = sending.find_next(class_='courseTitle')

                if conjunction:
                    if len(articulation.find_all('awc-view-conjunction')) > 1:
                        print('complex conjunction detected')
                    conjunction_text = conjunction.find_next('div').text.strip()
                    second_sending_num = conjunction.find_next(class_='prefixCourseNumber')
                    second_sending_title = conjunction.find_next(class_='courseTitle')
                    if conjunction_text == 'Or':
                        print(f'{receiving_course_num.text.strip()} {receiving_course_title.text.strip()}: '
                              f'{first_sending_num.text.strip()} {first_sending_title.text.strip()}, '
                              f'{second_sending_num.text.strip()} {second_sending_title.text.strip()}')
                    else:
                        print(f'{receiving_course_num.text.strip()} {receiving_course_title.text.strip()}: '
                              f'{first_sending_num.text.strip()} {first_sending_title.text.strip()}/'
                              f'{second_sending_num.text.strip()} {second_sending_title.text.strip()}')
                else:
                    print(f'{receiving_course_num.text.strip()} {receiving_course_title.text.strip()}: '
                          f'{first_sending_num.text.strip()} {first_sending_title.text.strip()}')

        await browser.close()

await scrape()

MATH 1A Calculus: MATH 3A Analytic Geometry and Calculus I, MATH 3AH Analytic Geometry and Calculus I Honors
MATH 1B Calculus: MATH 3B Analytic Geometry and Calculus II, MATH 3BH Analytic Geometry and Calculus II Honors
MATH 53 Multivariable Calculus: MATH 4A Analytic Geometry and Calculus III
complex conjunction detected
MATH 54 Linear Algebra and Differential Equations: MATH 24 Elementary Differential Equations/MATH 26 Introduction to Linear Algebra
complex conjunction detected
ECON 1 Introduction to Economics: ECON 1 Principles of Economics - Micro/ECON 2 Principles of Economics - Macro
ECON C3 Introduction to Environmental Economics and Policy: ENV 6 Environmental and Resource Economics
UGBA 10X Foundations of Business: MGT 1 Introduction to Business, MGT 1H Introduction to Business Honors
ENGIN 7 Introduction to Computer Programming for Scientists and Engineers (MATLAB): ENGR 20 Introduction to Programming and Problem Solving with MATLAB
